In [2]:
!python  -V

Python 3.9.12


In [84]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import pickle

## Load Data (Parquet)


In [33]:
data = pd.read_parquet("./data/green_tripdata_2021-01.parquet")

In [34]:
data.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,None,-0.3,-52.80,3.0,1.0,0.00
5,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,52.0,0.0,0.5,0.00,0.0,None,0.3,52.80,2.0,1.0,0.00
6,2,2021-01-01 00:19:14,2021-01-01 00:19:21,N,5.0,265,265,1.0,0.00,180.0,0.0,0.0,36.06,0.0,None,0.3,216.36,1.0,2.0,0.00
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,0.5,0.5,0.96,0.0,None,0.3,5.76,1.0,1.0,0.00
8,2,2021-01-01 00:57:46,2021-01-01 00:57:57,N,1.0,225,225,1.0,0.00,2.5,0.5,0.5,0.00,0.0,None,0.3,3.80,2.0,1.0,0.00
9,2,2021-01-01 00:58:32,2021-01-01 01:32:34,N,1.0,225,265,1.0,12.19,38.0,0.5,0.5,2.75,0.0,None,0.3,42.05,1.0,1.0,0.00


## Feature Engineering

In [40]:
# Calculate Trip-time
data["duration"] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime

# convert to minutes
data.duration = data.duration.apply(lambda dur: dur.total_seconds()/60)

# focus on data that is less than 60min and more than 1min
data = data[(data.duration >= 1) & (data.duration <= 60)]

# Take a subset
# data = data[data.trip_type == 2]

In [47]:
# which variables to take?
categorical = ["PULocationID", "DOLocationID"]
numerical = ["duration"]

data = data[categorical + numerical]

# do one hot encoding for categorical features
data[categorical] = data[categorical].astype(str)

In [50]:
# convert df to dictionaries
train_dicts = data.to_dict(orient="records")

dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)

In [57]:
# Set the Target
target = "duration"
y_train = data[target].values

## Create a function

In [65]:
def process_data(filename):
    data = pd.read_parquet(filename)
    
    data["duration"] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data.duration = data.duration.apply(lambda dur: dur.total_seconds()/60)
    data = data[(data.duration >= 1) & (data.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]

    # do one hot encoding for categorical features
    data[categorical] = data[categorical].astype(str)
    
    return data

In [76]:
# Load train and validation data
train_d = process_data("./data/green_tripdata_2021-01.parquet")
val_d = process_data("./data/green_tripdata_2021-02.parquet")

In [77]:
train_d.shape, val_d.shape

((73908, 21), (61921, 21))

In [78]:
# create new feature
train_d["PU_DO"] = train_d["PULocationID"] + "-" + train_d["DOLocationID"]
val_d["PU_DO"] = val_d["PULocationID"] + "-" + val_d["DOLocationID"]

In [79]:
# create DictVectorizer
dv = DictVectorizer()
categorical = ["PU_DO"]
numerical = ["trip_distance"]

train_dicts = train_d[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = val_d[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [80]:
target = "duration"
Y_train = train_d[target].values
Y_val = val_d[target].values

## Training + Validation

In [83]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_val)

mean_squared_error(Y_val, Y_pred, squared=False)

7.479603461126994

In [85]:
# Save Model
with open("./models/lr-reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)